In [1]:
from keras.datasets import cifar10
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.transforms import ToTensor
import torchvision.models as models
from PIL import Image
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sb

from functools import partial
from collections import OrderedDict

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import time
import pickle

In [2]:
hidden_width = 32
hidden_nblocks = 4
train_max_epoch = 30
max_iterations = 10

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'
L2_param = 1e-5

label_name = "visible_ice"
num_classes = 10
output_size = 10

sm = nn.Softmax(dim=-1)

In [3]:
(trainX, trainY), (testX, testY) = cifar10.load_data()

In [4]:
print('Train: X=%s, y=%s' % (trainX.shape, trainY.shape))
print('Test: X=%s, y=%s' % (testX.shape, testY.shape))

Train: X=(50000, 32, 32, 3), y=(50000, 1)
Test: X=(10000, 32, 32, 3), y=(10000, 1)


In [5]:
number_of_rows = trainX.shape[0]
random_indices = np.random.choice(number_of_rows, size=number_of_rows//100, replace=False)
trainX = trainX[random_indices]
trainY = trainY[random_indices]

In [6]:
trainX.shape

(500, 32, 32, 3)

In [7]:
class cifar10_Dataset(Dataset):
    def __init__(self, image_data, labels):
        
        
        self.tabular_data = np.empty([labels.shape[0], 5])
        self.image_data = image_data
        self.labels = labels
        # cargo capacity, number of legs, number of wheels, organic, water
        for i, label in enumerate(labels):
            if label[0] == 0: # airplane
                self.tabular_data[i] = [1000, 0, 10, 0, 0]
            elif label[0] == 1: # automobile
                self.tabular_data[i] = [200, 0, 4, 0, 0]
            elif label[0] == 2: # bird
                self.tabular_data[i] = [0, 2, 0, 1, 0]
            elif label[0] == 3: # cat
                self.tabular_data[i] = [0, 4, 0, 1, 0]
            elif label[0] == 4: # deer
                self.tabular_data[i] = [0, 4, 0, 1, 0]
            elif label[0] == 5: # dog
                self.tabular_data[i] = [0, 4, 0, 1, 1]
            elif label[0] == 6: # frog
                self.tabular_data[i] = [0, 4, 0, 1, 1]
            elif label[0] == 7: #horse
                self.tabular_data[i] = [100, 4, 0, 1, 0]
            elif label[0] == 8: # ship
                self.tabular_data[i] = [5000, 0, 0, 0, 1]
            elif label[0] == 9: #truck
                self.tabular_data[i] = [500, 0, 4, 0, 0]
        
        print('Dataset initialized')
        
    def __len__(self):
        return self.labels.shape[0]

    def __getitem__(self, idx):
        image = torch.tensor(self.image_data[idx]).float()
        image = torch.permute(image, (2,0,1))
        
        return {'tabular': torch.tensor(self.tabular_data[idx]).float(),'image': image,'label': torch.tensor(self.labels[idx][0]).long()}

In [8]:
train_dataset = cifar10_Dataset(trainX, trainY)
test_dataset = cifar10_Dataset(testX, testY)

Dataset initialized
Dataset initialized


In [9]:
# train_size = int(0.8 * len(full_dataset))
# test_size = len(full_dataset) - train_size

# batchsize can cause error when last leftover batchsize is 1, batchnorm cannot function on 1 sample data
batchsize = 20
# while(train_size % batchsize == 1):
#     batchsize+=1
# print(batchsize)

# train_data, test_data = torch.utils.data.random_split(full_dataset, [train_size, test_size], generator=torch.Generator().manual_seed(42))

trainloader = DataLoader(train_dataset, batch_size=batchsize, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=batchsize, shuffle=True)

## MLP

In [10]:
class mlp_pure(nn.Module):
        def __init__(self, input_size, output_size):
            super(mlp_pure, self).__init__()
            self.input_size = input_size
            self.output_size = output_size
            
            self.hidden_size = hidden_width
            self.hidden_nblocks = hidden_nblocks
            
            self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
            self.fc2 = torch.nn.Linear(self.hidden_size,self.hidden_size)
            self.fc3 = torch.nn.Linear(self.hidden_size, self.output_size)
            
            self.relu = torch.nn.ReLU()
            self.end = torch.nn.Softmax(dim=-1) ## sigmoid for multi-label, softmax for multi-class (mutually exclusive)
            
        def forward(self, x):
            out = self.fc1(x)
            out = self.relu(out)
            # print(out.shape)
            
            for i in range(self.hidden_nblocks):
                out = self.fc2(out)
                out = self.relu(out)
            
            out = self.fc3(out)
            # out = self.end(out)
            return out

In [11]:
def train_mlp(trainloader, testloader, print_epochs = False, loss_fn = torch.nn.BCELoss()):
    
    input_size = list(train_dataset[0]['tabular'].size())
    
    mlp_model = mlp_pure(input_size[0],output_size)
    mlp_model.to(device)
    
    optimizer = torch.optim.Adam(mlp_model.parameters(), weight_decay = L2_param)

    epoch_loss = np.zeros([train_max_epoch, 2])
    
    for epoch in range(train_max_epoch):  # loop over the dataset multiple times

        mlp_model.train()
        running_loss_sum = 0.0
        for i, data in enumerate(trainloader, 0): # loop over each sample

            # get the inputs; data is a list of [inputs, labels]
            tabular_data, labels = data['tabular'].to(device), data['label'].to(device)

            predicted = mlp_model(tabular_data)
            
            # print(predicted.shape)
            
            # squeeze: return tensor with all dimensions of size 1 removed
#             print(predicted.squeeze().shape)
#             print(labels.shape)
            
            loss = loss_fn(predicted.squeeze(), labels)

            optimizer.zero_grad()

            loss.backward()

            optimizer.step()

            running_loss_sum += loss.item()

        # ----------- get validation loss for current epoch --------------
        mlp_model.eval()
        validation_loss_sum = 0.0
        for i, data in enumerate(testloader, 0): # loop over each sample

            tabular_data, labels = data['tabular'].to(device), data['label'].to(device)
            predicted = mlp_model(tabular_data)
            
            loss = loss_fn(predicted.squeeze(), labels)

            validation_loss_sum += loss.item()

        # ---------------- print statistics ------------------------

        running_loss = running_loss_sum / len(trainloader)
        validation_loss = validation_loss_sum / len(testloader)
        epoch_loss[epoch, :] =  [running_loss, validation_loss]
        
        if print_epochs:
            print('epoch %2d: running loss: %.5f, validation loss: %.5f' %
                          (epoch + 1, running_loss, validation_loss))

        torch.save(mlp_model.state_dict(), os.path.join('mlp_test_models/', 'epoch-{}.pt'.format(epoch+1)))
    
    if print_epochs:
        print('Finished Training')
        
    return epoch_loss
  

In [12]:
      
def test_mlp(epoch_loss, print_model_epoch = False):
    
    # ------ select model ---------
    ind = np.argmin(epoch_loss[:, 1])
    
    input_size = list(train_dataset[0]['tabular'].size())
    
    mlp_model = mlp_pure(input_size[0],output_size)
    mlp_model.load_state_dict(torch.load('mlp_test_models/epoch-{}.pt'.format(ind+1)))
    
    mlp_model.to(device)
    
    if print_model_epoch:
        print("epoch {} model selected".format(ind+1))
    
    # evaluate model on test set
    mlp_model.eval()

    with torch.no_grad():
        y_test = []
        y_pred = []
        y_cert = []
        for i, data in enumerate(testloader, 0):
            tabular_data, labels = data['tabular'].to(device), data['label'].to(device)

            # y_test.append(label.numpy().list())
            # print(label.shape)
            # print(images.shape)

            output = mlp_model(tabular_data)
            
            output = sm(output)
            # print(output)
            
            
            max_results = torch.max(output, dim= -1)
            predicted = max_results.indices
            certainty = max_results.values
            #predicted = torch.round(output)
            # print(predicted.shape)
            
            y_test.extend(labels.tolist())
            y_pred.extend(predicted.tolist())
            y_cert.extend(certainty.tolist())
            


            
    
#     print(y_test)
#     print(y_pred)
#     with open("mlp-certainty/iteration_{}.txt".format(it), "wb") as fp:   #Pickling
#         pickle.dump(y_cert, fp)
    #with open("test.txt", "rb") as fp:   # Unpickling
        #b = pickle.load(fp)
    
    arr_accuracy = accuracy_score(y_test, y_pred)
    scores = precision_recall_fscore_support(y_test, y_pred, average=None, zero_division=0)
    return arr_accuracy, scores

## film

In [13]:
class mlp(nn.Module):
        def __init__(self, input_size, output_size = 1, hidden_width = 20, hidden_nblocks = 2):
            super(mlp, self).__init__()
            self.input_size = input_size
            self.output_size = output_size
            
            self.hidden_width = hidden_width
            self.hidden_nblocks = hidden_nblocks
            
            self.fc1 = nn.Linear(self.input_size, self.hidden_width)
            self.fc2 = nn.Linear(self.hidden_width,self.hidden_width)
            self.fc3 = nn.Linear(self.hidden_width, self.output_size)
            
            self.relu = torch.nn.ReLU()
            self.end= torch.nn.Softmax(dim = -1) ## sigmoid for multi-label, softmax for multi-class (mutually exclusive)
            
            self.dropout = nn.Dropout(0.25)
            
        def forward(self, x, film_params):
            out = self.fc1(x)
            out = self.relu(out)
            
            
            for i in range(self.hidden_nblocks):
                out = self.fc2(out)
                
                # ------- film layer -----------
                start = i * hidden_width * 2
                mid = start + hidden_width
                end = mid + hidden_width
                
                gamma = film_params[:, start : mid]
                beta = film_params[:, mid : end]
                
#                 print(out.shape)
#                 print(gamma.shape)
#                 print(beta.shape)
                
                out = out * gamma
                out += beta
                # ------- film layer -----------
                # out = self.dropout(out)
                out = self.relu(out)
            
            out = self.fc3(out)
            # out = self.end(out)
            return out

In [14]:
n_film_params = hidden_width * hidden_nblocks * 2

n_channels = 3

def train_model(trainloader, testloader, print_epochs = False, loss_fn = torch.nn.BCELoss()):
    # loss: binary cross entropy

    generator = models.resnet18()
    generator.fc = nn.Linear(512, n_film_params)
    generator.conv1 = nn.Conv2d(n_channels, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    
    gen_model = generator

    # print(gen_model)

    input_size = list(train_dataset[0]['tabular'].size())
    net_model = mlp(input_size[0], output_size, hidden_width, hidden_nblocks).to(device)
    
    gen_optimizer = torch.optim.Adam(gen_model.parameters(), weight_decay = L2_param)
    net_optimizer = torch.optim.Adam(net_model.parameters(), weight_decay = L2_param)
    
    gen_model.to(device)
    net_model.to(device)

    # --------- check back propagation ----------- -
    # net_model.fc1.weight.register_hook(lambda x: print('grad accumulated in mlp fc1'))
    # gen_first_layer = gen_model.encoder.blocks[0].blocks[0].blocks[0].conv
    # gen_first_layer.weight.register_hook(lambda x: print('grad accumulated in resnet first layer'))

    epoch_loss = np.zeros([train_max_epoch, 2])
    
    for epoch in range(train_max_epoch):  # loop over the dataset multiple times

        # ------------ train -----------------
        gen_model.train()
        net_model.train()
        running_loss_sum = 0.0
        for i, data in enumerate(trainloader, 0): # loop over each sample

            # get the inputs; data is a list of [inputs, labels]
            images, tabular_data, labels = data['image'].to(device), data['tabular'].to(device), data['label'].to(device)

            
            gen_params = gen_model(images)
            predicted = net_model(tabular_data, gen_params)
            predicted = torch.squeeze(predicted)
            loss = loss_fn(predicted, labels)

            gen_optimizer.zero_grad()
            net_optimizer.zero_grad()

            loss.backward()

            gen_optimizer.step()
            net_optimizer.step()

            running_loss_sum += loss.item()

        # ----------- get validation loss for current epoch --------------
        gen_model.eval()
        net_model.eval()
        validation_loss_sum = 0.0
        for i, data in enumerate(testloader, 0): # loop over each sample

            # get the inputs; data is a list of [inputs, labels]
            images, tabular_data, labels = data['image'].to(device), data['tabular'].to(device), data['label'].to(device)

            # TODO: exammine film_params gradients / readup pytorch
            gen_params = gen_model(images)
            predicted = net_model(tabular_data, gen_params)
            predicted = torch.squeeze(predicted)
            loss = loss_fn(predicted, labels)
            validation_loss_sum += loss.item()

        # ---------------- print statistics ------------------------

        running_loss = running_loss_sum / len(trainloader)
        validation_loss = validation_loss_sum / len(testloader)
        epoch_loss[epoch, :] =  [running_loss, validation_loss]
        
        if print_epochs:
            print('epoch %2d: running loss: %.5f, validation loss: %.5f' %
                          (epoch + 1, running_loss, validation_loss))

        torch.save(gen_model.state_dict(), os.path.join('film_test_models/', 'gen-epoch-{}.pt'.format(epoch+1)))
        torch.save(net_model.state_dict(), os.path.join('film_test_models/', 'net-epoch-{}.pt'.format(epoch+1)))

    if print_epochs:
        print('Finished Training')
    
    return epoch_loss

## Test model

def test_model(epoch_loss, print_model_epoch = False):
    
    # ------ select model ---------
    ind = np.argmin(epoch_loss[:, 1])
    
    generator = models.resnet18()
    generator.fc = nn.Linear(512, n_film_params)
    generator.conv1 = nn.Conv2d(n_channels, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    
    gen_model = generator

    input_size = list(train_dataset[0]['tabular'].size())
    net_model = mlp(input_size[0], output_size, hidden_width, hidden_nblocks)

    gen_model.load_state_dict(torch.load('film_test_models/gen-epoch-{}.pt'.format(ind+1)))
    net_model.load_state_dict(torch.load('film_test_models/net-epoch-{}.pt'.format(ind+1)))
    
    gen_model.to(device)
    net_model.to(device)
    
    if print_model_epoch:
        print("epoch {} model selected".format(ind+1))
    
    # evaluate model on test set
    gen_model.eval()
    net_model.eval()
    with torch.no_grad():
        y_test = []
        y_pred = []
        for i, data in enumerate(testloader, 0):
            images, tabular_data, labels = data['image'].to(device), data['tabular'].to(device), data['label'].to(device)

            # y_test.append(label.numpy().list())
            # print(label.shape)
            # print(images.shape)


            gen_params = gen_model(images)
            output = net_model(tabular_data, gen_params)
            
            
            max_results = torch.max(output, dim= -1)
            predicted = max_results.indices

            y_test.extend(labels.tolist())
            y_pred.extend(predicted.tolist())
#             predicted = torch.squeeze(predicted)
            
            
            
#             predicted = torch.round(predicted)
#             # print(predicted.shape)
#             lb = labels.tolist()
#             pr = predicted.tolist()
#             y_test.extend(lb)
#             y_pred.extend(pr)
    
    arr_accuracy = accuracy_score(y_test, y_pred)
    scores = precision_recall_fscore_support(y_test, y_pred, average=None, zero_division=0)
    return arr_accuracy, scores


#     print(confusion_matrix(y_test,y_pred))
#     print(classification_report(y_test,y_pred))
#     print(accuracy_score(y_test, y_pred))

## Pure MLP

In [18]:
max_iterations = 2
results = np.zeros([max_iterations, num_classes*4 + 1])
# trainloader, testloader = prepare_dataloader(full_dataset)

for it in range(max_iterations):
    start = time.time()
    
    # mlp
#     epoch_loss_mlp = train_mlp(trainloader,testloader, print_epochs = True, loss_fn = nn.CrossEntropyLoss())
#     acc, scores = test_mlp(epoch_loss_mlp, print_model_epoch = True)
    
    #     # ------- mlp-resnet film 
    epoch_loss = train_model(trainloader, testloader, print_epochs=True, loss_fn = nn.CrossEntropyLoss())
    acc, scores = test_model(epoch_loss, print_model_epoch = True)
    
    # scores = precision, recall, fscore, support
    results[it, 0] = acc
    
    for j, score in enumerate(scores):
        start_ind = 1 + j*num_classes
        results[it, start_ind: start_ind + num_classes] = score

    end = time.time()
    
    print('iteration {} elapsed time: {}, accuracy : {}'.format(it+1, end-start, acc))

c:\users\mouju\appdata\local\programs\python\python38\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


epoch  1: running loss: 3.82016, validation loss: 1511.83480
epoch  2: running loss: 3.36674, validation loss: 2.20859
epoch  3: running loss: 1.84525, validation loss: 3.33905
epoch  4: running loss: 1.89570, validation loss: 1.83640
epoch  5: running loss: 1.57828, validation loss: 1.53344
epoch  6: running loss: 1.44709, validation loss: 1.38590
epoch  7: running loss: 1.30745, validation loss: 1.17459
epoch  8: running loss: 1.10829, validation loss: 1.07993
epoch  9: running loss: 0.95354, validation loss: 1.04543
epoch 10: running loss: 0.95695, validation loss: 1.47651
epoch 11: running loss: 1.11262, validation loss: 1.28142
epoch 12: running loss: 1.29895, validation loss: 1.30649
epoch 13: running loss: 1.22276, validation loss: 1.19594
epoch 14: running loss: 0.83456, validation loss: 0.92169
epoch 15: running loss: 0.59939, validation loss: 0.87190
epoch 16: running loss: 0.70287, validation loss: 1.48153
epoch 17: running loss: 0.77317, validation loss: 0.85644
epoch 18: r

In [19]:
def display_table(scores):
    df = np.reshape(scores, [num_classes,4], order ='F')
    df = pd.DataFrame(df)
    
    # df.style.set_table_attributes("style='display:inline'").set_caption(mode)
    
    df.columns = ['precision', 'recall', 'f1', 'support']
    # df.index = ['unfrozen', 'frozen']
    # df.index = ['Visible ice', 'No visible ice']
    
    display(df)
    
def display_results(results):
    mean = np.mean(results, axis=0)
    std = np.std(results, axis=0)
    
    print("mean")
    display_table(mean[1:])
    
    print("std")
    display_table(std[1:])
    
    print("Accuracy mean: {}, std: {}".format(mean[0], std[0]))

display_results(results)

mean


,precision,recall,f1,support
0,0.942761,0.9780,0.959844,1000.0
1,0.986935,0.9835,0.985214,1000.0
2,0.944840,0.9990,0.970309,1000.0
3,0.688882,0.5555,0.595770,1000.0
4,0.644839,0.7475,0.682368,1000.0
5,0.802765,0.6125,0.690385,1000.0
6,0.680926,0.7925,0.727814,1000.0
7,0.992983,0.9920,0.992490,1000.0
8,0.983559,1.0000,0.991642,1000.0
9,0.983960,0.9335,0.957551,1000.0


std


,precision,recall,f1,support
0,0.049303,0.0220,0.036172,0.0
1,0.013065,0.0145,0.013785,0.0
2,0.055160,0.0010,0.028690,0.0
3,0.006356,0.1905,0.117082,0.0
4,0.075733,0.0925,0.003855,0.0
5,0.058346,0.0545,0.013201,0.0
6,0.007849,0.1175,0.045996,0.0
7,0.005019,0.0070,0.006011,0.0
8,0.016441,0.0000,0.008358,0.0
9,0.016040,0.0585,0.038433,0.0


Accuracy mean: 0.8593999999999999, std: 0.005799999999999972
